In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# Topic: EX2 - Turbofan RUL Prediction
**Task**: Predict the remaining useful life (RUL) of turbofan engines based on given sensor data (time series data). It is a regression problem.
**Data**: Turbofan engine degradation simulation data (NASA) - [Link](https://data.nasa.gov/dataset/Turbofan-Engine-Degradation-Simulation-Data-Set/vrks-gjie). See also in the topic [introduction notebook](https://github.com/nina-prog/damage-propagation-modeling/blob/2fb8c1a1102a48d7abbf04e4031807790a913a99/notebooks/Turbofan%20remaining%20useful%20life%20Prediction.ipynb).

**Subtasks**:
1. Perform a deep **exploratory data analysis (EDA)** on the given data.
2. Implement a more efficient **sliding window method** for time series data analysis. -> 🎯 **Focus on this task**
3. Apply **traditional machine learning methods** (SOTA) to predict the remaining useful life. Includes data preparation, feature extraction, feature selection, model selection, and model parameter optimization.
4. Create **neural network models** to predict the remaining useful life. Includes different architectures like Convolutional Neural Networks (CNN), Recurrent Neural Networks (RNN), or Attention Models. Note: You can search for SOTA research papers and reproduce current state-of-the-art models.


# Imports + Settings

In [1]:
# third-party libraries
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt

from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series, impute
from tsfresh.feature_extraction import MinimalFCParameters

In [3]:
# source code
from src.data_loading import load_data, load_config
from src.rolling_window_creator import calculate_RUL

In [5]:
# settings
sns.set_style("whitegrid")
sns.set_palette("Set2")
sns.set(rc={"figure.dpi":100, 'savefig.dpi':200})
sns.set_context('notebook')

In [6]:
np.random.seed(42)

# Paths

In [7]:
os.chdir("../") # set working directory to root of project
#os.getcwd() # check current working directory

In [8]:
PATH_TO_CONFIG = "configs/config.yaml"

# Load Config + Data

In [9]:
config = load_config(PATH_TO_CONFIG) # config is dict

In [10]:
train_data, test_data, test_RUL_data = load_data(config_path=PATH_TO_CONFIG, dataset_num=1)

2024-05-30 22:46:01 [src.data_loading:43] [INFO] >>>> Loading data set 1...
2024-05-30 22:46:01 [src.data_loading:72] [INFO] >>>> Loaded raw data for dataset 1.
2024-05-30 22:46:01 [src.data_loading:73] [INFO] >>>> Train Data: (20631, 26)
2024-05-30 22:46:01 [src.data_loading:74] [INFO] >>>> Test Data: (13096, 26)
2024-05-30 22:46:01 [src.data_loading:75] [INFO] >>>> Test RUL Data: (100, 1)


# 📍 Subtask 2: Sliding Window Method

---
# Variant 1: Linear Degradation Model

Note:
* In training however, we need multiple examples to train. If we would only use the time series until today we would only have a single training example. Therefore, we use a trick: we replay the history.
* At each time step $t$, you treat the data as it would be today. You extract the features with everything you know until today (which is all data until and including $t$). The target for the features until time $t$ is the time value of time t (which we already know, because everything has already happened).
* The process of window-sliding is implemented in the function `roll_time_series`. Our window size will be 20 (we look at max 20 days in the past) and we disregard all windows which are shorter than 5 days.

Generating Data to train involves the following steps:
1. Generate rolling windows of size k (e.g. 20) from the time series data. It means that we take the last k time periods as relevant features, including the current time period. The target is then the one of the last included time step (no forcasting in our case!).
2. Extract features from the rolling windows. This can be as simple as taking the mean of the last k days, or more complex features like the slope of a linear regression. --> The data would then be of size (n_samples/k, n_features) and the target (n_samples/k, 1), this for every potential subgroup

Visualization of the applied sliding window approach:
<img src="../images/sliding_window_variant1.jpg" alt="Visualization of the applied sliding window approach" width="1000"/>


## 1. Generate rolling windows

In [23]:
# define window size
# with min_k every window with fewer values than this are not used
MAX_K = 20
MIN_K = 20

Given a k=20 the roll time series function also includes the k+1 data, so we use a downgrade of -1 for the timeshift parameter. It has to be used in this function because the maximum timeshift when expecting a window of size 20 (max_window_size) has to be 19, since the current cycle is also included in the window and counts as a time step itself in the window.

In [24]:
%%time
# generate rolling windows
train_data_rolled = roll_time_series(train_data, column_id="UnitNumber", column_sort="Cycle", max_timeshift=MAX_K-1, min_timeshift=MIN_K-1)

Rolling: 100%|██████████| 37/37 [00:04<00:00,  7.56it/s]


CPU times: total: 359 ms
Wall time: 5.32 s


In [25]:
train_data_rolled.head(5)

,UnitNumber,Cycle,Operation Setting 1,Operation Setting 2,Operation Setting 3,Sensor Measure 1,Sensor Measure 2,Sensor Measure 3,Sensor Measure 4,Sensor Measure 5,...,Sensor Measure 13,Sensor Measure 14,Sensor Measure 15,Sensor Measure 16,Sensor Measure 17,Sensor Measure 18,Sensor Measure 19,Sensor Measure 20,Sensor Measure 21,id
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,"(1, 20)"
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,"(1, 20)"
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442,"(1, 20)"
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739,"(1, 20)"
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044,"(1, 20)"


In [38]:
# count number of windows to be generated, by logic
num_units = train_data["UnitNumber"].nunique()
window_count = 0
for i in range(1, num_units+1):
    # number of possible "window starting points" for each unit
    window_count += train_data[train_data["UnitNumber"] == i].shape[0] - MAX_K + 1
print(f"Number of windows to be generated: {window_count}")
# actual number of windows generated
print(f"Number of windows actually generated: {train_data_rolled['id'].nunique()}")

Number of windows to be generated: 18731
Number of windows actually generated: 18731


Findings:
* **Function Result**: New column `id` is generated, representing a **window id**. It is a tuple of the **`UnitNumber` (certain group)** and the **ending `Cycle` (ending time step)**.
* **Example**: Data with the `id` (1, 40) contains the original data of `UnitNumber` 1 of the last 20 `Cycle` steps including `Cycle` 40.

In [14]:
# check if example interpretation is correct and if the window is generated correctly
rolled_win = train_data_rolled[train_data_rolled["id"] == (1, 40)]
print(rolled_win.shape)
display(rolled_win.head(5))

original_win_data = train_data[(train_data["UnitNumber"] == 1) &
                               (train_data["Cycle"] <= 40) &
                               (train_data["Cycle"] >= 40 - 20)]
print(original_win_data.shape)
display(original_win_data.head(5))

(20, 27)


,UnitNumber,Cycle,Operation Setting 1,Operation Setting 2,Operation Setting 3,Sensor Measure 1,Sensor Measure 2,Sensor Measure 3,Sensor Measure 4,Sensor Measure 5,...,Sensor Measure 13,Sensor Measure 14,Sensor Measure 15,Sensor Measure 16,Sensor Measure 17,Sensor Measure 18,Sensor Measure 19,Sensor Measure 20,Sensor Measure 21,id
62000,1,21,-0.0012,0.0001,100.0,518.67,642.37,1586.07,1398.13,14.62,...,2388.08,8134.02,8.4049,0.03,392,2388,100.0,39.09,23.3101,"(1, 40)"
62001,1,22,0.0002,0.0000,100.0,518.67,642.77,1592.93,1400.57,14.62,...,2388.03,8130.41,8.4034,0.03,392,2388,100.0,38.92,23.3792,"(1, 40)"
62002,1,23,0.0034,-0.0003,100.0,518.67,642.14,1588.19,1394.75,14.62,...,2388.05,8127.90,8.4240,0.03,392,2388,100.0,38.94,23.4562,"(1, 40)"
62003,1,24,-0.0010,0.0003,100.0,518.67,642.38,1590.83,1398.81,14.62,...,2388.03,8133.88,8.3891,0.03,392,2388,100.0,39.00,23.3696,"(1, 40)"
62004,1,25,0.0023,-0.0004,100.0,518.67,642.77,1594.10,1399.39,14.62,...,2388.02,8136.61,8.3917,0.03,393,2388,100.0,38.95,23.4288,"(1, 40)"


(21, 26)


,UnitNumber,Cycle,Operation Setting 1,Operation Setting 2,Operation Setting 3,Sensor Measure 1,Sensor Measure 2,Sensor Measure 3,Sensor Measure 4,Sensor Measure 5,...,Sensor Measure 12,Sensor Measure 13,Sensor Measure 14,Sensor Measure 15,Sensor Measure 16,Sensor Measure 17,Sensor Measure 18,Sensor Measure 19,Sensor Measure 20,Sensor Measure 21
19,1,20,-0.0037,0.0001,100.0,518.67,643.04,1581.11,1405.23,14.62,...,522.07,2388.02,8129.71,8.4210,0.03,392,2388,100.0,39.03,23.4220
20,1,21,-0.0012,0.0001,100.0,518.67,642.37,1586.07,1398.13,14.62,...,522.42,2388.08,8134.02,8.4049,0.03,392,2388,100.0,39.09,23.3101
21,1,22,0.0002,0.0000,100.0,518.67,642.77,1592.93,1400.57,14.62,...,522.00,2388.03,8130.41,8.4034,0.03,392,2388,100.0,38.92,23.3792
22,1,23,0.0034,-0.0003,100.0,518.67,642.14,1588.19,1394.75,14.62,...,521.52,2388.05,8127.90,8.4240,0.03,392,2388,100.0,38.94,23.4562
23,1,24,-0.0010,0.0003,100.0,518.67,642.38,1590.83,1398.81,14.62,...,522.13,2388.03,8133.88,8.3891,0.03,392,2388,100.0,39.00,23.3696


In [15]:
print(f"Number of samples: {train_data_rolled.shape[0]}")
print(f"Number of unique windows: {train_data_rolled['id'].nunique()}")
print(f"Number of windows shorter than 5 days: {train_data_rolled['id'].value_counts().loc[lambda x: x < MIN_K].sum()}")
print(f"Number of unique units: {train_data_rolled['UnitNumber'].nunique()}")

Number of samples: 392120
Number of unique windows: 20131
Number of windows shorter than 5 days: 0
Number of unique units: 100


In [16]:
# check windows sizes / lengths
train_data_rolled.groupby("id").size().value_counts()

20    18731
6       100
7       100
8       100
9       100
10      100
11      100
12      100
13      100
14      100
15      100
16      100
17      100
18      100
19      100
Name: count, dtype: int64

In [17]:
# get list of features
features = config["dataloading"]["features"]["operational_settings"] + \
           config["dataloading"]["features"]["sensor_measure"]

Findings:
* The fact that there are always a number of 100 windows for each window size less than 20 and more than 5 indicates that there are 100 unique units in the data and that the windows are generated correctly.

## 2. Extract Features
Note:
* We have many features and the `RUL` column is the target. so we thus need to extract features for each feature column in the window and then also get a y value for each window.

In [18]:
%%time
# extract features - tsfresh
X = extract_features(train_data_rolled.drop(["UnitNumber"], axis=1),
                     column_id="id", column_sort="Cycle",
                     default_fc_parameters=MinimalFCParameters(),
                     impute_function=impute, show_warnings=False)
# add index names
X.index = X.index.rename(["UnitNumber", "Cycle"])

Feature Extraction: 100%|██████████| 40/40 [01:16<00:00,  1.91s/it]


CPU times: total: 30.9 s
Wall time: 1min 27s


In [19]:
X.head(5)

Operation Setting 1__sum_values  \
UnitNumber Cycle                                    
1          6                              -0.0086   
           7                              -0.0076   
           8                              -0.0110   
           9                              -0.0102   
           10                             -0.0135   

                  Operation Setting 1__median  Operation Setting 1__mean  \
UnitNumber Cycle                                                           
1          6                          -0.0013                  -0.001433   
           7                          -0.0007                  -0.001086   
           8                          -0.0013                  -0.001375   
           9                          -0.0007                  -0.001133   
           10                         -0.0013                  -0.001350   

                  Operation Setting 1__length  \
UnitNumber Cycle                                
1          6                              6.0   
           7                              7.0   
           8                              8.0   
           9                              9.0   
           10                            10.0   

                  Operation Setting 1__standard_deviation  \
UnitNumber Cycle                                            
1          6                                     0.002340   
           7                                     0.002328   
           8                                     0.002308   
           9                                     0.002281   
           10                                    0.002259   

                  Operation Setting 1__variance  \
UnitNumber Cycle                                  
1          6                           0.000005   
           7                           0.000005   
           8                           0.000005   
           9                           0.000005   
           10                          0.000005   

                  Operation Setting 1__root_mean_square  \
UnitNumber Cycle                                          
1          6                                   0.002744   
           7                                   0.002568   
           8                                   0.002687   
           9                                   0.002547   
           10                                  0.002632   

                  Operation Setting 1__maximum  \
UnitNumber Cycle                                 
1          6                            0.0019   
           7                            0.0019   
           8                            0.0019   
           9                            0.0019   
           10                           0.0019   

                  Operation Setting 1__absolute_maximum  \
UnitNumber Cycle                                          
1          6                                     0.0043   
           7                                     0.0043   
           8                                     0.0043   
           9                                     0.0043   
           10                                    0.0043   

                  Operation Setting 1__minimum  ...  \
UnitNumber Cycle                                ...   
1          6                           -0.0043  ...   
           7                           -0.0043  ...   
           8                           -0.0043  ...   
           9                           -0.0043  ...   
           10                          -0.0043  ...   

                  Sensor Measure 21__sum_values  Sensor Measure 21__median  \
UnitNumber Cycle                                                             
1          6                           140.3320                   23.38915   
           7                           163.7094                   23.37740   
           8                           187.0200                   23.37565   
           9                

In [20]:
# check if example window can also be found in the extracted features
X.loc[(1, 40)]

Operation Setting 1__sum_values            -0.007000
Operation Setting 1__median                -0.000200
Operation Setting 1__mean                  -0.000350
Operation Setting 1__length                20.000000
Operation Setting 1__standard_deviation     0.001804
                                             ...    
Sensor Measure 21__variance                 0.003237
Sensor Measure 21__root_mean_square        23.393159
Sensor Measure 21__maximum                 23.499900
Sensor Measure 21__absolute_maximum        23.499900
Sensor Measure 21__minimum                 23.284100
Name: (1, 40), Length: 240, dtype: float64

Potential Features:
* Uisng tsfresh Comprehensive Set:
    * Contains around **64** feature calculators and hundreds of resulting features. Each calculator can produce multiple features with different parameters.
* Using tsfresh Efficient Set:
    * Includes fewer calculators and parameters, focusing on features that are quicker to compute. It typically includes around **40-50** unique feature calculators.
* Using tsfresh Minimal Set:
    * The minimal set includes the following **8** basic features:
        * "absolute_maximum"
        * "absolute_minimum"
        * "mean"
        * "median"
        * "standard_deviation"
        * "variance"
        * "skewness"
        * "kurtosis"
* Using Custom Set:
    * Custom set of features can be defined by the user. In our case a set is defined as a subset from all possible tsfresh features available. This can be done by defining a dictionary with the feature names and the corresponding parameters. It is specifically defined in the config file.
    * We selected the basics as well as basic features with some parameter variations. Also, some additional features that also capture the trend of the data and the distribution of the data.
    * The custom set contains **48** features.

The process is selected by defining the `feature_extraction_mode` in the `RollingWindowDatasetCreator` class. The default is set to `minimal`. Other options are `efficient` and `comprehensive`, as well as a `custom` dictionary with the feature names and the corresponding parameters.

## 3. Extract Target & Map to Features
Note:
* **Target Example**: The target for the row with the id (1, 40) is the RUL of the row with the id (1, 40) meaning unit number 1, cycle 40

In [21]:
# calculate RUL
train_data_RUL = calculate_RUL(data=train_data, time_column="Cycle", group_column="UnitNumber")

In [22]:
train_data_RUL[(train_data_RUL["UnitNumber"] == 1)].tail(5)

,UnitNumber,Cycle,Operation Setting 1,Operation Setting 2,Operation Setting 3,Sensor Measure 1,Sensor Measure 2,Sensor Measure 3,Sensor Measure 4,Sensor Measure 5,...,Sensor Measure 13,Sensor Measure 14,Sensor Measure 15,Sensor Measure 16,Sensor Measure 17,Sensor Measure 18,Sensor Measure 19,Sensor Measure 20,Sensor Measure 21,RUL
187,1,188,-0.0067,0.0003,100.0,518.67,643.75,1602.38,1422.78,14.62,...,2388.23,8117.69,8.5207,0.03,396,2388,100.0,38.51,22.9588,5
188,1,189,-0.0006,0.0002,100.0,518.67,644.18,1596.17,1428.01,14.62,...,2388.33,8117.51,8.5183,0.03,395,2388,100.0,38.48,23.1127,4
189,1,190,-0.0027,0.0001,100.0,518.67,643.64,1599.22,1425.95,14.62,...,2388.35,8112.58,8.5223,0.03,398,2388,100.0,38.49,23.0675,3
190,1,191,-0.0000,-0.0004,100.0,518.67,643.34,1602.36,1425.77,14.62,...,2388.30,8114.61,8.5174,0.03,394,2388,100.0,38.45,23.1295,2
191,1,192,0.0009,-0.0000,100.0,518.67,643.54,1601.41,1427.20,14.62,...,2388.32,8110.93,8.5113,0.03,396,2388,100.0,38.48,22.9649,1


In [23]:
# extract target
y = train_data_RUL.set_index(["UnitNumber", "Cycle"]).sort_index().RUL

In [43]:
# consistency check for example window (1, 40)
print(f"y value for window (1, 40): {y.loc[(1, 40)]}") # should be the same as the RUL of the row with the id (1, 40)
print(f"Train data RUL value for UnitNumber, Cycle (1, 40): {train_data_RUL[(train_data_RUL['UnitNumber'] == 1) & (train_data_RUL['Cycle'] == 40)]['RUL'].values[0]}")

y value for window (1, 40): 153
Train data RUL value for UnitNumber, Cycle (1, 40): 153


Findings:
* The target value RUL does indeed match the RUL of the last cycle in the window.

In [25]:
# compare index and size of X and y
print(f"Shape of X: {X.shape}")
print(f"Fist 3 rows of index of X: {X.index[:3]}")
print(f"Shape of y: {y.shape}")
print(f"Fist 3 rows of index of y: {y.index[:3]}")

Shape of X: (20131, 240)
Fist 3 rows of index of X: MultiIndex([(1, 6),
            (1, 7),
            (1, 8)],
           names=['UnitNumber', 'Cycle'])
Shape of y: (20631,)
Fist 3 rows of index of y: MultiIndex([(1, 1),
            (1, 2),
            (1, 3)],
           names=['UnitNumber', 'Cycle'])


In [26]:
# check last rows of y
y.tail(5)

UnitNumber  Cycle
100         196      5
            197      4
            198      3
            199      2
            200      1
Name: RUL, dtype: int64

In [27]:
# show nan values in y
print(f"Number of NaN values in y: {y.isnull().sum()}")
y[y.isnull()]

Number of NaN values in y: 0


Series([], Name: RUL, dtype: int64)

Findings:

There are some inconsistencies! :
* X is missing the first 5 dates (as our minimum window size was 5), which can be seen looking at the index length of X and y. X index has a length of `20131` and y index has a length of `20631`. This `500` difference is because we have `100` units and the first `5` cycles of each unit are not in X, thus we have $100 * 5 = 500$ missing rows in X which are in y. --> drop the first min_k time steps / cycles from X

In [28]:
# make X and y consistent
y = y[y.index.isin(X.index)]
X = X[X.index.isin(y.index)]

In [29]:
# compare index of X and y
print(f"Shape of X: {X.shape}")
print(f"Fist 3 rows of index of X: {X.index[:3]}")
print(f"Shape of y: {y.shape}")
print(f"Fist 3 rows of index of y: {y.index[:3]}")

Shape of X: (20131, 240)
Fist 3 rows of index of X: MultiIndex([(1, 6),
            (1, 7),
            (1, 8)],
           names=['UnitNumber', 'Cycle'])
Shape of y: (20131,)
Fist 3 rows of index of y: MultiIndex([(1, 6),
            (1, 7),
            (1, 8)],
           names=['UnitNumber', 'Cycle'])


In [30]:
print(f"Shape of X: {X.shape}")
display(X.tail(5))
print(f"Shape of y: {y.shape}")
display(y.tail(5))

Shape of X: (20131, 240)


Operation Setting 1__sum_values  \
UnitNumber Cycle                                    
100        196                             0.0052   
           197                             0.0047   
           198                             0.0046   
           199                             0.0015   
           200                            -0.0007   

                  Operation Setting 1__median  Operation Setting 1__mean  \
UnitNumber Cycle                                                           
100        196                       -0.00015                   0.000260   
           197                       -0.00015                   0.000235   
           198                       -0.00015                   0.000230   
           199                       -0.00030                   0.000075   
           200                       -0.00030                  -0.000035   

                  Operation Setting 1__length  \
UnitNumber Cycle                                
100        196                           20.0   
           197                           20.0   
           198                           20.0   
           199                           20.0   
           200                           20.0   

                  Operation Setting 1__standard_deviation  \
UnitNumber Cycle                                            
100        196                                   0.001346   
           197                                   0.001375   
           198                                   0.001374   
           199                                   0.001340   
           200                                   0.001504   

                  Operation Setting 1__variance  \
UnitNumber Cycle                                  
100        196                         0.000002   
           197                         0.000002   
           198                         0.000002   
           199                         0.000002   
           200                         0.000002   

                  Operation Setting 1__root_mean_square  \
UnitNumber Cycle                                          
100        196                                 0.001370   
           197                                 0.001395   
           198                                 0.001393   
           199                                 0.001342   
           200                                 0.001504   

                  Operation Setting 1__maximum  \
UnitNumber Cycle                                 
100        196                          0.0027   
           197                          0.0027   
           198                          0.0027   
           199                          0.0027   
           200                          0.0027   

                  Operation Setting 1__absolute_maximum  \
UnitNumber Cycle                                          
100        196                                   0.0027   
           197                                   0.0027   
           198                                   0.0027   
           199                                   0.0027   
           200                                   0.0032   

                  Operation Setting 1__minimum  ...  \
UnitNumber Cycle                                ...   
100        196                         -0.0014  ...   
           197                         -0.0016  ...   
           198                         -0.0016  ...   
           199                         -0.0016  ...   
           200                         -0.0032  ...   

                  Sensor Measure 21__sum_values  Sensor Measure 21__median  \
UnitNumber Cycle                                                             
100        196                         462.4189                   23.12235   
           197                         462.5431                   23.12590   
           198                         462.2512                   23.12235   
           199              

Shape of y: (20131,)


UnitNumber  Cycle
100         196      5
            197      4
            198      3
            199      2
            200      1
Name: RUL, dtype: int64

## 3. Modification for Test Set
Note:
* The test is different from the training set, as we have extern RUL target values for the test set. This target consits only of one value for each unit, which is the RUL of the last cycle of each unit. Thus, we need to filter the data.
* Only use last window of each unit in the test set. The target RUL is in the `test_RUL_data` dataframe, which is the true RUL of the last cycle of each unit, thus a shape of (100, 1).

--> Filter rolled data to only include the last window of each UnitNumber:
* Ensure the selected cycles are sorted in ascending order of UnitNumber and Cycle values.
* Group the filtered data by UnitNumber.
* For each UnitNumber group, select the last max_window_size cycles.


In [31]:
# generate rolling windows for test data (-1 due to the same already mentioned in the train set)
test_data_rolled = roll_time_series(test_data, column_id="UnitNumber", column_sort="Cycle", max_timeshift=MAX_K-1, min_timeshift=MIN_K)

Rolling: 100%|██████████| 38/38 [00:04<00:00,  8.84it/s]


In [32]:
test_data_rolled.head(5)

,UnitNumber,Cycle,Operation Setting 1,Operation Setting 2,Operation Setting 3,Sensor Measure 1,Sensor Measure 2,Sensor Measure 3,Sensor Measure 4,Sensor Measure 5,...,Sensor Measure 13,Sensor Measure 14,Sensor Measure 15,Sensor Measure 16,Sensor Measure 17,Sensor Measure 18,Sensor Measure 19,Sensor Measure 20,Sensor Measure 21,id
0,1,1,0.0023,0.0003,100.0,518.67,643.02,1585.29,1398.21,14.62,...,2388.03,8125.55,8.4052,0.03,392,2388,100.0,38.86,23.3735,"(1, 6)"
1,1,2,-0.0027,-0.0003,100.0,518.67,641.71,1588.45,1395.42,14.62,...,2388.06,8139.62,8.3803,0.03,393,2388,100.0,39.02,23.3916,"(1, 6)"
2,1,3,0.0003,0.0001,100.0,518.67,642.46,1586.94,1401.34,14.62,...,2388.03,8130.10,8.4441,0.03,393,2388,100.0,39.08,23.4166,"(1, 6)"
3,1,4,0.0042,0.0000,100.0,518.67,642.44,1584.12,1406.42,14.62,...,2388.05,8132.90,8.3917,0.03,391,2388,100.0,39.00,23.3737,"(1, 6)"
4,1,5,0.0014,0.0000,100.0,518.67,642.51,1587.19,1401.92,14.62,...,2388.03,8129.54,8.4031,0.03,390,2388,100.0,38.99,23.4130,"(1, 6)"


In [33]:
# filter to only include the last window of each unit
filtered_test_data_rolled = test_data_rolled.groupby("UnitNumber").tail(MAX_K)

In [34]:
filtered_test_data_rolled.head(40)

,UnitNumber,Cycle,Operation Setting 1,Operation Setting 2,Operation Setting 3,Sensor Measure 1,Sensor Measure 2,Sensor Measure 3,Sensor Measure 4,Sensor Measure 5,...,Sensor Measure 13,Sensor Measure 14,Sensor Measure 15,Sensor Measure 16,Sensor Measure 17,Sensor Measure 18,Sensor Measure 19,Sensor Measure 20,Sensor Measure 21,id
39420,1,12,0.0026,0.0003,100.0,518.67,642.54,1587.43,1397.82,14.62,...,2388.06,8132.33,8.3984,0.03,391,2388,100.0,39.11,23.3845,"(1, 31)"
39421,1,13,-0.0056,0.0003,100.0,518.67,641.94,1589.09,1403.94,14.62,...,2388.03,8131.12,8.4166,0.03,392,2388,100.0,39.08,23.3677,"(1, 31)"
39422,1,14,0.0017,-0.0004,100.0,518.67,642.23,1583.16,1402.88,14.62,...,2388.06,8130.30,8.4293,0.03,392,2388,100.0,39.03,23.4572,"(1, 31)"
39423,1,15,-0.0003,-0.0003,100.0,518.67,642.50,1584.81,1398.79,14.62,...,2388.00,8133.62,8.4163,0.03,392,2388,100.0,39.04,23.3672,"(1, 31)"
39424,1,16,-0.0018,0.0003,100.0,518.67,642.32,1584.51,1407.76,14.62,...,2388.10,8133.83,8.4300,0.03,390,2388,100.0,38.87,23.3484,"(1, 31)"
39425,1,17,0.0014,0.0002,100.0,518.67,642.19,1582.70,1404.12,14.62,...,2388.02,8126.78,8.4577,0.03,391,2388,100.0,39.09,23.3409,"(1, 31)"
39426,1,18,0.0035,0.0001,100.0,518.67,642.59,1586.53,1403.69,14.62,...,2388.06,8133.22,8.4323,0.03,391,2388,100.0,38.96,23.4481,"(1, 31)"
39427,1,19,0.0029,0.0001,100.0,518.67,642.43,1585.58,1402.30,14.62,...,2388.01,8129.31,8.3892,0.03,391,2388,100.0,39.06,23.3809,"(1, 31)"
39428,1,20,0.0011,-0.0001,100.0,518.67,642.61,1587.78,1400.70,14.62,...,2388.05,8128.59,8.4099,0.03,392,2388,100.0,39.00,23.3325,"(1, 31)"
39429,1,21,0.0038,-0.0002,100.0,518.67,642.70,1583.30,1399.20,14.62,...,2388.11,8126.86,8.4174,0.03,392,2388,100.0,38.96,23.4025,"(1, 31)"


In [35]:
# extract features for test data
X_test = extract_features(filtered_test_data_rolled.drop(["UnitNumber"], axis=1),
                          column_id="id", column_sort="Cycle",
                          default_fc_parameters=MinimalFCParameters(),
                          impute_function=impute, show_warnings=False)
# add index names
X_test.index = X_test.index.rename(["UnitNumber", "Cycle"])

Feature Extraction: 100%|██████████| 40/40 [00:03<00:00, 13.29it/s]


In [36]:
# extract target for test data - match index of y_test with X_test
y_test = test_RUL_data
y_test.index = X_test.index

In [37]:
# check if X_test and y_test match
print(f"Shape of X_test: {X_test.shape}")
display(X_test.head(5))
print(f"Shape of y_test: {y_test.shape}")
display(y_test.head(5))

Shape of X_test: (100, 240)


,,Operation Setting 1__sum_values,Operation Setting 1__median,Operation Setting 1__mean,Operation Setting 1__length,Operation Setting 1__standard_deviation,Operation Setting 1__variance,Operation Setting 1__root_mean_square,Operation Setting 1__maximum,Operation Setting 1__absolute_maximum,Operation Setting 1__minimum,...,Sensor Measure 21__sum_values,Sensor Measure 21__median,Sensor Measure 21__mean,Sensor Measure 21__length,Sensor Measure 21__standard_deviation,Sensor Measure 21__variance,Sensor Measure 21__root_mean_square,Sensor Measure 21__maximum,Sensor Measure 21__absolute_maximum,Sensor Measure 21__minimum
UnitNumber,Cycle,,,,,,,,,,,,,,,,,,,,,
1,31,1.810000e-02,0.00130,9.050000e-04,20.0,0.002373,0.000006,0.002540,0.0047,0.0056,-0.0056,...,467.5243,23.37235,23.376215,20.0,0.034147,0.001166,23.376240,23.4572,23.4572,23.3186
2,49,-9.100000e-03,0.00005,-4.550000e-04,20.0,0.002212,0.000005,0.002258,0.0032,0.0039,-0.0039,...,465.9096,23.30065,23.295480,20.0,0.041742,0.001742,23.295517,23.3693,23.3693,23.2196
3,126,-4.400000e-03,-0.00025,-2.200000e-04,20.0,0.002824,0.000008,0.002832,0.0057,0.0057,-0.0057,...,465.0032,23.24990,23.250160,20.0,0.043298,0.001875,23.250200,23.3559,23.3559,23.1749
4,106,-4.336809e-19,0.00005,-2.168404e-20,20.0,0.001972,0.000004,0.001972,0.0031,0.0048,-0.0048,...,465.5171,23.27205,23.275855,20.0,0.056855,0.003232,23.275924,23.3769,23.3769,23.1676
5,98,1.640000e-02,0.00105,8.200000e-04,20.0,0.002243,0.000005,0.002388,0.0062,0.0062,-0.0035,...,465.1504,23.24325,23.257520,20.0,0.064874,0.004209,23.257610,23.4117,23.4117,23.1476


Shape of y_test: (100, 1)


,,RUL
UnitNumber,Cycle,
1,31,112
2,49,98
3,126,69
4,106,82
5,98,91


## 4. Summarize in Function

In [38]:
from src.rolling_window_creator import RollingWindowDatasetCreator

In [39]:
creator = RollingWindowDatasetCreator(column_id="UnitNumber",
                                      column_sort="Cycle",
                                      max_window_size=config["preprocessing"]["max_window_size"],
                                      min_window_size=config["preprocessing"]["min_window_size"],
                                      feature_extraction_mode='minimal',
                                      early_RUL=None)
X_train, y_train, X_test, y_test = creator.create_rolling_windows_datasets(train_data, test_data, test_RUL_data)

2024-05-30 12:09:59 [src.rolling_window_creator:117] [INFO] >>>> Creating rolling windows for train data...


Rolling: 100%|██████████| 37/37 [00:05<00:00,  7.29it/s]


2024-05-30 12:10:04 [src.rolling_window_creator:125] [INFO] >>>> Extracting features for train data...


Feature Extraction: 100%|██████████| 40/40 [01:03<00:00,  1.58s/it]


2024-05-30 12:11:19 [src.rolling_window_creator:133] [INFO] >>>> Calculating target for train data...
2024-05-30 12:11:19 [src.rolling_window_creator:117] [INFO] >>>> Creating rolling windows for test data...


Rolling: 100%|██████████| 38/38 [00:04<00:00,  9.40it/s]


2024-05-30 12:11:24 [src.rolling_window_creator:125] [INFO] >>>> Extracting features for test data...


Feature Extraction: 100%|██████████| 40/40 [00:02<00:00, 13.65it/s]


2024-05-30 12:11:27 [src.rolling_window_creator:161] [INFO] >>>> Datasets created successfully.
2024-05-30 12:11:27 [src.rolling_window_creator:162] [INFO] >>>> Shape of X_train: (20131, 240)
2024-05-30 12:11:27 [src.rolling_window_creator:163] [INFO] >>>> Shape of y_train: (20131, 1)
2024-05-30 12:11:27 [src.rolling_window_creator:164] [INFO] >>>> Shape of X_test: (100, 240)
2024-05-30 12:11:27 [src.rolling_window_creator:165] [INFO] >>>> Shape of y_test: (100, 1)


In [40]:
# check if X_train and y_train match
print(f"Shape of X_train: {X_train.shape}")
display(X_train.head(5))
print(f"Shape of y_train: {y_train.shape}")
display(y_train.head(5))

Shape of X_train: (20131, 240)


Operation Setting 1__sum_values  \
UnitNumber Cycle                                    
1          6                              -0.0086   
           7                              -0.0076   
           8                              -0.0110   
           9                              -0.0102   
           10                             -0.0135   

                  Operation Setting 1__median  Operation Setting 1__mean  \
UnitNumber Cycle                                                           
1          6                          -0.0013                  -0.001433   
           7                          -0.0007                  -0.001086   
           8                          -0.0013                  -0.001375   
           9                          -0.0007                  -0.001133   
           10                         -0.0013                  -0.001350   

                  Operation Setting 1__length  \
UnitNumber Cycle                                
1          6                              6.0   
           7                              7.0   
           8                              8.0   
           9                              9.0   
           10                            10.0   

                  Operation Setting 1__standard_deviation  \
UnitNumber Cycle                                            
1          6                                     0.002340   
           7                                     0.002328   
           8                                     0.002308   
           9                                     0.002281   
           10                                    0.002259   

                  Operation Setting 1__variance  \
UnitNumber Cycle                                  
1          6                           0.000005   
           7                           0.000005   
           8                           0.000005   
           9                           0.000005   
           10                          0.000005   

                  Operation Setting 1__root_mean_square  \
UnitNumber Cycle                                          
1          6                                   0.002744   
           7                                   0.002568   
           8                                   0.002687   
           9                                   0.002547   
           10                                  0.002632   

                  Operation Setting 1__maximum  \
UnitNumber Cycle                                 
1          6                            0.0019   
           7                            0.0019   
           8                            0.0019   
           9                            0.0019   
           10                           0.0019   

                  Operation Setting 1__absolute_maximum  \
UnitNumber Cycle                                          
1          6                                     0.0043   
           7                                     0.0043   
           8                                     0.0043   
           9                                     0.0043   
           10                                    0.0043   

                  Operation Setting 1__minimum  ...  \
UnitNumber Cycle                                ...   
1          6                           -0.0043  ...   
           7                           -0.0043  ...   
           8                           -0.0043  ...   
           9                           -0.0043  ...   
           10                          -0.0043  ...   

                  Sensor Measure 21__sum_values  Sensor Measure 21__median  \
UnitNumber Cycle                                                             
1          6                           140.3320                   23.38915   
           7                           163.7094                   23.37740   
           8                           187.0200                   23.37565   
           9                

Shape of y_train: (20131, 1)


RUL
UnitNumber Cycle     
1          6      187
           7      186
           8      185
           9      184
           10     183

In [41]:
# check if X_test and y_test match
print(f"Shape of X_test: {X_test.shape}")
display(X_test.head(5))
print(f"Shape of y_test: {y_test.shape}")
display(y_test.head(5))

Shape of X_test: (100, 240)


,,Sensor Measure 10__sum_values,Sensor Measure 10__median,Sensor Measure 10__mean,Sensor Measure 10__length,Sensor Measure 10__standard_deviation,Sensor Measure 10__variance,Sensor Measure 10__root_mean_square,Sensor Measure 10__maximum,Sensor Measure 10__absolute_maximum,Sensor Measure 10__minimum,...,Sensor Measure 9__sum_values,Sensor Measure 9__median,Sensor Measure 9__mean,Sensor Measure 9__length,Sensor Measure 9__standard_deviation,Sensor Measure 9__variance,Sensor Measure 9__root_mean_square,Sensor Measure 9__maximum,Sensor Measure 9__absolute_maximum,Sensor Measure 9__minimum
UnitNumber,Cycle,,,,,,,,,,,,,,,,,,,,,
1,31,19.5,1.3,1.3,15.0,2.220446e-16,4.930381e-32,1.3,1.3,1.3,1.3,...,135726.40,9046.90,9048.426667,15.0,4.519551,20.426342,9048.427795,9056.40,9056.40,9041.12
2,49,19.5,1.3,1.3,15.0,2.220446e-16,4.930381e-32,1.3,1.3,1.3,1.3,...,135719.03,9048.35,9047.935333,15.0,2.786065,7.762158,9047.935762,9052.32,9052.32,9042.03
3,126,19.5,1.3,1.3,15.0,2.220446e-16,4.930381e-32,1.3,1.3,1.3,1.3,...,135756.92,9049.41,9050.461333,15.0,4.376840,19.156732,9050.462392,9058.16,9058.16,9044.93
4,106,19.5,1.3,1.3,15.0,2.220446e-16,4.930381e-32,1.3,1.3,1.3,1.3,...,135812.17,9054.42,9054.144667,15.0,4.782503,22.872332,9054.145930,9062.04,9062.04,9046.96
5,98,19.5,1.3,1.3,15.0,2.220446e-16,4.930381e-32,1.3,1.3,1.3,1.3,...,135791.72,9053.85,9052.781333,15.0,3.870707,14.982372,9052.782161,9059.29,9059.29,9044.76


Shape of y_test: (100, 1)


,,RUL
UnitNumber,Cycle,
1,31,112
2,49,98
3,126,69
4,106,82
5,98,91


In [42]:
# save processed data (as pickle)
#timestamp = time.strftime("%Y%m%d-%H%M%S")
#X_train.to_pickle(f"{config['paths']['processed_data_dir']}ex2_X_train_{timestamp}.pkl")
#y_train.to_pickle(f"{config['paths']['processed_data_dir']}ex2_y_train_{timestamp}.pkl")
#X_test.to_pickle(f"{config['paths']['processed_data_dir']}ex2_X_test_{timestamp}.pkl")
#y_test.to_pickle(f"{config['paths']['processed_data_dir']}ex2_y_test_{timestamp}.pkl")


---
# Variant 2: Piecewise Linear Degradation Model

The piecewise linear degradation model is a more realistic model for the degradation of turbofan engines. It is based on the assumption that the degradation of the engine is not linear over the whole life cycle, but rather consists of several linear segments. In this context there is an early RUL, which is a constant. This is set until the actual RUL is reached, then the degradation is linear. The early RUL is a hyperparameter that can be set. The piecewise and linear degradation model is implemented in the `calculate_RUL` function in the `rolling_window_creator.py` file.

Example: If the early RUL is set to 100, the RUL is 100 until the actual RUL reaches 100, then the degradation is linear. For implementation, we can calculate the RUL for each unit with a linear degradation model, but we afterward clip the RUL at a certain value (early RUL). This is done in the `calculate_RUL` function.

In [2]:
# calculate pice wise and linear degradation of RUL
clipping_val = 150

picewise_deg = calculate_RUL(train_data, time_column="Cycle", group_column="UnitNumber", clipping_value_of_RUL=clipping_val)
lin_deg = calculate_RUL(train_data, time_column="Cycle", group_column="UnitNumber", clipping_value_of_RUL=None)

NameError: name 'train_data' is not defined

In [ ]:
# plot pice wise and linear degradation of RUL for i random units
i = 4

num_cols = 2
num_rows = int(np.ceil(i / num_cols))
units = np.random.choice(picewise_deg["UnitNumber"].unique(), i, replace=False)

fig, ax = plt.subplots(num_rows, num_cols, figsize=(12, 10))
for idx, unit in enumerate(units):
    # plot piecewise and linear degradation
    sns.lineplot(data=picewise_deg[picewise_deg["UnitNumber"]==unit], x="Cycle", y="RUL", ax=ax[idx//num_cols, idx%num_cols], label=f"Piecewise Degradation ({clipping_val} clipped)")
    # plot linear degradation
    sns.lineplot(data=lin_deg[lin_deg["UnitNumber"]==unit], x="Cycle", y="RUL", ax=ax[idx//num_cols, idx%num_cols], label="Linear Degradation")
    # set title, labels and intersection point
    ax[idx//num_cols, idx%num_cols].set_title(f"Unit {unit}")
    ax[idx//num_cols, idx%num_cols].set_xlabel("Cycle")
    ax[idx//num_cols, idx%num_cols].set_ylabel("RUL")
    # get first intersection point of both degradation curves
    intersection_point = picewise_deg[picewise_deg["UnitNumber"]==unit].merge(lin_deg[lin_deg["UnitNumber"]==unit], on="Cycle", suffixes=("_piecewise", "_linear"))
    intersection_point = intersection_point[intersection_point["RUL_piecewise"] == intersection_point["RUL_linear"]].iloc[0]
    # plot intersection point  and add vertical line up to the intersection point
    ax[idx//num_cols, idx%num_cols].scatter(intersection_point["Cycle"], intersection_point["RUL_piecewise"], color="red", label=f"Intersection Point ({int(intersection_point['Cycle'])}, {int(intersection_point['RUL_piecewise'])})")
    ymax_fraction = intersection_point["RUL_piecewise"] / ax[idx//num_cols, idx%num_cols].get_ylim()[1]
    ax[idx//num_cols, idx%num_cols].axvline(intersection_point["Cycle"], color="red", linestyle="--", ymax=ymax_fraction)
    ax[idx//num_cols, idx%num_cols].legend()
plt.suptitle("Comparison of Piecewise and Linear Degradation of RUL for Random Units")
plt.tight_layout()
plt.show()

Findings:
* The picewise linear degradation model shows the expected behavior. The RUL is constant until the early RUL is reached, then the degradation is linear.
* Can be seen in the plots where the intersection point of the two degradation curves is the point where the early RUL is reached and the linear degradation starts.
